<a href="https://colab.research.google.com/github/iamonk/Algorithmic_Trading/blob/main/Equal_weight_S%26P_500/Equal_weight_S%26P_500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [4]:
import numpy as np
import pandas as pd
import math
import requests

First we will install XlsxWriter (COLAB)

In [2]:
!pip install XlsxWriter

     |████████████████████████████████| 153kB 5.4MB/s 


In [5]:
import xlsxwriter

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [4]:
from google.colab import files
upload=files.upload()

Saving sp_500_stocks.csv to sp_500_stocks.csv


In [12]:
stocks=pd.read_csv("sp_500_stocks.csv")

In [17]:
stocks.columns

Index(['Ticker'], dtype='object')

Process of importing and reading Csv files from googledrive.

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

Upload Files

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
!cp gdrive/MyDrive/secrets.py secrets.py

In [10]:
!ls

gdrive	sample_data  secrets.py  sp_500_stocks.csv


In [1]:
from secrets import IEX_CLOUD_API_TOKEN

In [ ]:
!ls

gdrive	__pycache__  sample_data  secrets.py  sp_500_stocks.csv


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



Every Api is different, check that api documentattion and there sites to know about there api GET POST PUT AND DELETE.

In [6]:
symbol = 'AAPL'
apl_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data=requests.get(apl_url).json()
data

{'avgTotalVolume': 97166210,
 'calculationPrice': 'close',
 'change': 0.27,
 'changePercent': 0.00224,
 'close': None,
 'closeSource': 'cfiifloa',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 127.2,
 'iexCloseTime': 1635189370582,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 126.35,
 'iexOpenTime': 1655347718541,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'isUSMarketOpen': False,
 'lastTradeTime': 1625099093304,
 'latestPrice': 122.77,
 'latestSource': 'Close',
 'latestTime': 'February 26, 2021',
 'latestUpdate': 1651760442628,
 'latestVolume': None,
 'low': None,
 'lowSource': None,
 'lowTime': None,
 'marketCap': 2068277662

## How TO USE Fstring(This is me)

In [ ]:
adj="Superb"
adj2="cool"
adj3="Strong"
string=f"Anurag is {adj},{adj2} and {adj3}"
string

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [7]:
price=data['latestPrice']
market_cap=data['marketCap']/1000000000000
price

122.77

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [8]:
my_columns=['Ticker','Stock Price','Market Capitalization','Number of Shares to Buy']
final_dataframe=pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [9]:
final_dataframe.append(
    pd.Series(
        [symbol,
         price,
         market_cap,
         'N/A'],index=my_columns
    ),ignore_index=True)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,122.77,2.068278,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [ ]:
final_dataframe=pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    apl_url=f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(apl_url).json()
    final_dataframe=final_dataframe.append(
        pd.Series(
            [
             stock,
             data['latestPrice'],
             data['marketCap'],
             'N/A' 
            ],index=my_columns
        ),ignore_index=True
    )

NameError: ignored

In [ ]:
final_dataframe


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,122.73,38896004741,N/A
1,AAL,21.46,13556203093,N/A
2,AAP,163.03,10767650951,N/A
3,AAPL,122.42,2090655463389,N/A
4,ABBV,111.84,191733096566,N/A
...,...,...,...,...
500,YUM,105.62,31533283734,N/A
501,ZBH,164.94,35041604041,N/A
502,ZBRA,501.90,27662940391,N/A
503,ZION,53.74,8811805904,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [14]:
def chunks(lst, n):
    """Yield successive n-sized chunks from list."""
    for i in range(0,len(lst),n):
        yield lst[i:i+n]
        

In [15]:
symbol_groups=list(chunks(stocks['Ticker'],100))

In [16]:
symbol_groups[1]

100     CME
101     CMG
102     CMI
103     CMS
104     CNC
       ... 
195    FTNT
196     FTV
197      GD
198      GE
199    GILD
Name: Ticker, Length: 100, dtype: object

In [19]:
symbol_strings=[]
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])
final_dataframe=pd.DataFrame(columns=my_columns)


for symbol_string in symbol_strings:
    batch_api_call_url=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe=final_dataframe.append(
            pd.Series([
                       symbol,
                       data[symbol]['quote']['latestPrice'],
                       data[symbol]['quote']['marketCap'],
                       'N/A'
            ],index=my_columns),ignore_index=True
        )
        

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [20]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,128.08,37656523849,N/A
1,AAL,21.56,13705781096,N/A
2,AAP,167.24,10929316520,N/A
3,AAPL,124.65,2060711879691,N/A
4,ABBV,111.51,196914721670,N/A
...,...,...,...,...
500,YUM,106.98,31135877801,N/A
501,ZBH,167.19,34266847336,N/A
502,ZBRA,522.06,26905765407,N/A
503,ZION,54.76,8887661864,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [21]:
print("SYMBOL\n\n")
print(symbol)
print("SYMBOL STRING\n\n")
print(symbol_string)
print("SYMBOL Group\n\n")
print(symbol_groups)
print("Symbol STRings\n\n")
print(symbol_strings)

SYMBOL


ZTS
SYMBOL STRING


YUM,ZBH,ZBRA,ZION,ZTS
SYMBOL Group


[0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
      ...  
95     CINF
96       CL
97      CLX
98      CMA
99    CMCSA
Name: Ticker, Length: 100, dtype: object, 100     CME
101     CMG
102     CMI
103     CMS
104     CNC
       ... 
195    FTNT
196     FTV
197      GD
198      GE
199    GILD
Name: Ticker, Length: 100, dtype: object, 200     GIS
201      GL
202     GLW
203      GM
204    GOOG
       ... 
295     MAA
296     MAR
297     MAS
298     MCD
299    MCHP
Name: Ticker, Length: 100, dtype: object, 300     MCK
301     MCO
302    MDLZ
303     MDT
304     MET
       ... 
395     RHI
396     RJF
397      RL
398     RMD
399     ROK
Name: Ticker, Length: 100, dtype: object, 400     ROL
401     ROP
402    ROST
403     RSG
404     RTX
       ... 
495    XLNX
496     XOM
497    XRAY
498     XRX
499     XYL
Name: Ticker, Length: 100, dtype: object, 500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Nam

In [24]:
portfolio_size=input('Enter the value of your portfolio:')

try:

    val=float(portfolio_size)
    print(val)
except ValueError:
    print('Thats Not a number \n Please try again')
    portfolio_size=input('Enter the value of your portfolio:') 
    val=float(portfolio_size) 

Enter the value of your portfolio:10000000
10000000.0


In [25]:
position_size=val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy']=math.floor(position_size/final_dataframe.loc[i,'Stock Price'])
final_dataframe    

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,128.08,37656523849,154
1,AAL,21.56,13705781096,918
2,AAP,167.24,10929316520,118
3,AAPL,124.65,2060711879691,158
4,ABBV,111.51,196914721670,177
...,...,...,...,...
500,YUM,106.98,31135877801,185
501,ZBH,167.19,34266847336,118
502,ZBRA,522.06,26905765407,37
503,ZION,54.76,8887661864,361


0

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [70]:
writer=pd.ExcelWriter('recommend trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades',index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [71]:
 background_color="#0a0a23"
 font_color="#ffffff"

 string_format=writer.book.add_format(
     {
      'font_color': font_color,
      'bg_color': background_color,
      'border': 1
     }
 )
 dollar_format=writer.book.add_format(
     {
      'num_format':'$0.00',   
      'font_color': font_color,
      'bg_color': background_color,
      'border': 1
     }
 )
 integer_format=writer.book.add_format(
     {'num_format':'0',
      'font_color': font_color,
      'bg_color': background_color,
      'border': 1
     }
 )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [72]:
#writer.sheets['Recommended Trades'].set_column('A:A',18,string_format)
#writer.sheets['Recommended Trades'].set_column('B:B',18,string_format)
#writer.sheets['Recommended Trades'].set_column('C:C',18,string_format)
#writer.sheets['Recommended Trades'].set_column('D:D',18,string_format)
#writer.save()

writer.sheets['Recommended Trades'].write('A1','Ticker',string_format)
writer.sheets['Recommended Trades'].write('B1','Stock Price',dollar_format)
writer.sheets['Recommended Trades'].write('C1','Market Capitalization',dollar_format)
writer.sheets['Recommended Trades'].write('D1','Number of Share to Buy',integer_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [74]:
!cp 'recommend trades.xlsx' gdrive/MyDrive/rc.xlsx

In [73]:
column_formats={
    'A':['Ticker', string_format],
    'B':['Stock Price', dollar_format],
    'C':['Market Capitalization', dollar_format],
    'D':['Number of Shares to Buy', integer_format]
}
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',18,column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0],column_formats[column][1])
writer.save()    

## Saving Our Excel Output

Saving our Excel file is very easy:

In [58]:
!ls

 gdrive       'recommend trades.xlsx'   secrets.py
 __pycache__   sample_data	        sp_500_stocks.csv


In [51]:
!rm 'recommend trades.xlsx'

In [ ]:
writer.save()